## Introduction to Bioinformatics - part 1  
This file includes some of the problems handled in the San Diego course of Bioinformatics (Coursera), in a form of consecutive  functions that can be used by later functions to solve more complex problems.  
The part 1 handles finding the origin of replication (ori) within a string of DNA, by multiple methods:  
    
    *Ori contains motifs (6-12 mer) that are repeated multiple times in the small segment of ori, which is not random, but indicates a purposful repetition of ori specific motifs that are recognized by replication enzymes. Therefore, we search for a narrow window that contains a suspiciously high motif repeats, and then count this repeats along with their complementary repeats (as the enzymes also bind to the template strand). Then, we take the motif with highest occurence considering a minimum mismatch rate by calculating hamming distances. However, some bacterial genomes have unusual patterns of repetition through the whole genome, rendering our search useless due to high rate of false positives.  
    
    *The other method is to calculate the GC-skew that results because of mutative events that happen on intermediate ssDNA during replication. The higher the skew, the further the position from ori (in a plasmid).

In [31]:
import re
def is_valid_dna(string):
    pattern = r'^[GCAT]*$'
    return re.match(pattern, string) is not None

DNA = "AAACACACATCATTGGGAGCATATCATCATACATAGACCATA"

### Pattern Counter  
a code used to count occurence of some short string (pattern) of DNA within a longer one (sequence).

In [19]:

def PatternCount(string, pattern):
    count = 0
    for i in range(len(string)-len(pattern)+1):
        if string[i:i+len(pattern)] == pattern:
            count = count+1
    return count

print(PatternCount(DNA,"CAT"))

7


### Frequency map:  
returns a dictionary with all k-mers with their frequency within a string of DNA

In [20]:
def FrequencyMap(string, k_len):      ## k_len is the length of the k-mer (like trimer or tertramer or dimer,etc)
    freq = {}
    n = len(string)
    for i in range(n-k_len+1):
        pattern = string[i:i+k_len]          ##assign each sliding window (by one base) to a key with a count value
        freq[pattern] = freq.get(pattern,0)+1  ###for each pattern: if that pattern is already present as a key, then get its value and add 1, otherwise just get 0 and add 1.
    return freq

print(FrequencyMap(DNA,3))

{'AAA': 1, 'AAC': 1, 'ACA': 4, 'CAC': 2, 'CAT': 7, 'ATC': 3, 'TCA': 3, 'ATT': 1, 'TTG': 1, 'TGG': 1, 'GGG': 1, 'GGA': 1, 'GAG': 1, 'AGC': 1, 'GCA': 1, 'ATA': 4, 'TAT': 1, 'TAC': 1, 'TAG': 1, 'AGA': 1, 'GAC': 1, 'ACC': 1, 'CCA': 1}


### Most frequent pattern  
Gets the most appearing string pattern(s) of length k in a DNA string.

In [30]:
def MostFrequent(string, k_len):
    most_freq_list = []
    freq_dict = FrequencyMap(string,k_len)
    max_freq = max(list(freq_dict.values()))
    for pattern, freq in freq_dict.items():
        if freq_dict[pattern] == max_freq:
            most_freq_list.append(pattern)
    return most_freq_list

print(MostFrequent(DNA,3))

['CAT']


### Complement Strand  

In [39]:

def complement(string):
    trans_dict = {"T":"A","A":"T","C":"G","G":"C"} #translation dictionary that replaces each base with complementary one
    map_table =  str.maketrans(trans_dict) ##make the mapping table with trans dict
    if is_valid_dna(string.rstrip().upper()):
        comp_strand = string.rstrip().upper().translate(map_table) ##replace using the mapping table within our string
    else:
        print("Warning: The string contains bases other than ACTG.")
    return comp_strand[::-1] ###reverse the complementary sequence to be from 5' to 3'

print(complement(DNA))

print(f"\nso the double strand would appear like:\n\n5'-{string}-3'\n3'-{complement(DNA)[::-1]}-5'")

TATGGTCTATGTATGATGATATGCTCCCAATGATGTGTGTTT

so the double strand would appear like:

5'-AAACACACATCATTGGGAGCATATCATCATACATAGACCATA-3'
3'-TTTGTGTGTAGTAACCCTCGTATAGTAGTATGTATCTGGTAT-5'


### Pattern matching positioner  
renders the position(s) of input patterns within a string.

In [43]:
def PatternMatching(pattern, genome):
    gen_len = len(genome)
    pat_len = len(pattern)
    positions = []
    for i in range(gen_len-pat_len+1):
        window = genome[i:i+pat_len]
        if window == pattern:
            positions.append(i)
    return positions

print(f"{PatternMatching('CAT',DNA)} by python indexing system (starting from 0)")

[7, 10, 19, 24, 27, 31, 38] by python indexing system (starting from 0)


--------

### The base score array  
This function allows for making a search and count window, and record the count (number of occurences of the pattern) for each window. The aim is to detect the positions that are having the highest occurences of that specific pattern.  
In our case we might be willing to seach for one specific nucleotide to study the GC-skew for example.

In [53]:
def BaseScoreArray(genome, base):
    score_array = {}
    n = len(genome)
    extended_genome = genome + genome[0:n//2]  ## We extend the genome (simulate that it is circular DNA of a plasmid)
    for i in range(n):
        score_array[i] = PatternCount(extended_genome[i:i+(n//2)],base) ## the count window is half the genome string, and slides one base at a time
    return score_array          ##the score of counts for each starting window's position are recorded in the score array

print(BaseScoreArray(DNA,"G"))


#### However, there is another, more computationaly effective way to do the bid

def FasterBaseScoreArray(genome, base):
    score_array = {}
    n = len(genome)
    extended_genome = genome + genome[0:n//2]         ##for initial steps we do the same preparation
    score_array[0] = PatternCount(genome[0:n//2], base)     ### But we count the base occurences only for the first window
    for i in range(1, n):                ## then for the following slides, we check only the "out" base and the "in" base (in other words, the base that was removed from the window because it slided, and the base that was added at the end of the window)
        score_array[i] = score_array[i-1]
        if extended_genome[i-1] == base:        ### if that "out" base matching our wanted base, then we make the score of that index equals the previous one - 1 because we lost one match in our window
            score_array[i] = score_array[i]-1
        if extended_genome[i+(n//2)-1] == base: ### if that newly "in" base matching our wanted base, then we make the score of that index equals the previous one + 1
            score_array[i] = score_array[i]+1
    return score_array

print(f"{FasterBaseScoreArray(DNA,'G')}, See... same result, but for an actual huge genome, more efficient!")

{0: 4, 1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 3, 17: 2, 18: 2, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 2, 38: 3, 39: 3, 40: 4, 41: 4}
{0: 4, 1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 4, 10: 4, 11: 4, 12: 4, 13: 4, 14: 4, 15: 4, 16: 3, 17: 2, 18: 2, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 2, 38: 3, 39: 3, 40: 4, 41: 4}, See... same result, but for an actual huge genome, more efficient!


### GC-Skew array   
One of the interesting properties of DNA replication is that DNA polymerases don't have a de novo sythesis ability, so it ends by making a leading strand and a lagging strand. If we take one strand of a circular DNA (plasmid), we can deduce that starting from the origin of replication (ori), one half will be a leading strand, and the other will be lagging. The leading strand is assumed to spend more time as single stranded (ssDNA), hence more prone to mutations and changes that disrupt the equilibrium of nucleotides (presumed) random distribution. Also, this will occur more often as we go far away from ori.  
One of the most common mutations happens to Cytosine (C) that transforms it into T (by deamination-methylation), resulting in a reduction of GC content. This is called "GC skew" and it is calculated as (G-C)/(G+C). So, the more mutated C there is, the bigger the gap is going to be between G and C (relative to the total GC). ori should be located at in the position with smallest GC-skew.

In [81]:
def SkewScoreArray(genome):
    skew_array = {}
    n = len(genome)
    extended_genome = genome + genome[0:n//2]
    skew_array[0] = 0                              ##We start at random in plasmid, and pick it as position 0 with 0 skew
    for i in range(0,n):
        if extended_genome[i] == "G":         ### to calculate according to the formula, we add up 1 for each G and add -1 for each C
            skew_array[i+1] = skew_array[i]+1
        if extended_genome[i] == "C":
            skew_array[i+1] = skew_array[i]-1
        if extended_genome[i] == "A" or extended_genome[i] == "T":
            skew_array[i+1] = skew_array[i]
    return skew_array

skewed_DNA = "CACCCCCCCCTGGAAAGATATATATATAGGAGGGAGGGGTGGGTTGGAGATATTTACCCATATACCACACATATTAATATATAGTACCAC"
print(SkewScoreArray(skewed_DNA))

{0: 0, 1: -1, 2: -1, 3: -2, 4: -3, 5: -4, 6: -5, 7: -6, 8: -7, 9: -8, 10: -9, 11: -9, 12: -8, 13: -7, 14: -7, 15: -7, 16: -7, 17: -6, 18: -6, 19: -6, 20: -6, 21: -6, 22: -6, 23: -6, 24: -6, 25: -6, 26: -6, 27: -6, 28: -6, 29: -5, 30: -4, 31: -4, 32: -3, 33: -2, 34: -1, 35: -1, 36: 0, 37: 1, 38: 2, 39: 3, 40: 3, 41: 4, 42: 5, 43: 6, 44: 6, 45: 6, 46: 7, 47: 8, 48: 8, 49: 9, 50: 9, 51: 9, 52: 9, 53: 9, 54: 9, 55: 9, 56: 9, 57: 8, 58: 7, 59: 6, 60: 6, 61: 6, 62: 6, 63: 6, 64: 6, 65: 5, 66: 4, 67: 4, 68: 3, 69: 3, 70: 2, 71: 2, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 2, 79: 2, 80: 2, 81: 2, 82: 2, 83: 2, 84: 3, 85: 3, 86: 3, 87: 2, 88: 1, 89: 1, 90: 0}


### Minimum Skew

In [90]:
def MinimumSkew(genome):
    skew_dict = SkewScoreArray(genome)
    return {k for k,v in skew_dict.items() if v == min(skew_dict.values())}  ## we take the key (position in DNA) if the value (the skew score) is the minimum of all skew scores.

print(MinimumSkew(skewed_DNA))

{10, 11}


### Hamming Distance  
The number of mismatches between two strings

In [91]:
def HammingDistance(seq1, seq2):
    count = 0
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            count += 1
    return count

print(HammingDistance("AGAGGTGT","TGACGTCT"))

3


### Approximate Pattern Matching  
A function to find locations of segments that hold a high similarity (lower than the difference limit) to the compared pattern. 

In [97]:
def ApproximatePatternMatching(String, Pattern, d): 
    positions = []
    for i in range(len(String)-len(Pattern)+1):
        if HammingDistance(String[i:i+len(Pattern)], Pattern) <= d:       ###while sliding a window with length equal to the pattern, check hamming distance of that window and the pattern. If it was lower than the mismatch limit, then accept it.
            positions.append(i)
    return positions

print(ApproximatePatternMatching(DNA,"CATATTC",2))  # the result contains the positions of segments that differ from the pattern by 2 bases or less.

[19, 31]


### Approximate Pattern Counter  
Counts the number of segments that are no different from the pattern by some threshold (d). 

In [106]:
def ApproximatePatternCount(String, Pattern, d):
    n = len(String)
    k = len(Pattern)
    count = 0
    for i in range(n-k+1):
        if HammingDistance(String[i:i+k],Pattern) <= d:
            count += 1
    return count

print(ApproximatePatternCount(DNA,"CATATTC",2))

### In the presence of the previous function, of course we can just say len(ApproximatePatternMatching(DNA,"CATATTC",2))


2
